In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# D

In [1]:
! pip install langchain tiktoken pypdf chromadb
! pip install transformers InstructorEmbedding sentence_transformers==2.2.2
! pip install accelerate bitsandbytes xformers
# ! pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.1 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 7.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.7/521.7 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 67.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings, HuggingFaceBgeEmbeddings

In [ ]:
file_loader = DirectoryLoader("/kaggle/input/thermal-pdfs", glob="./*.pdf", loader_cls=PyPDFLoader)
documents = file_loader.load()

In [ ]:
for doc in documents:
    doc.page_content = doc.page_content.replace("\n","")

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

In [ ]:
texts = text_splitter.split_documents(documents)

In [4]:
model_name = "BAAI/bge-large-en-v1.5"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cuda'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [5]:
persist_directory = "thermal"

In [6]:
# vector_db = Chroma.from_documents(documents=texts,
# #                                 embedding=instructor_embedding,
#                                   embedding=bge_embeddings,
#                                 persist_directory=persist_directory)
# vector_db.persist()

In [7]:
vector_db = Chroma(persist_directory=persist_directory, embedding_function=bge_embeddings)

In [118]:
retriever = vector_db.as_retriever(search_type="mmr",search_kwargs={"k":10})

In [2]:
#Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM, MistralForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta")
model = MistralForCausalLM.from_pretrained("HuggingFaceH4/zephyr-7b-beta",
                                            load_in_8bit=True,
                                              device_map='auto',
                                              torch_dtype=torch.float16,
                                              low_cpu_mem_usage=True,
#                                               attn_implementation="flash_attention_2",
                                              )

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [2]:
# from huggingface_hub import login
# login()

In [3]:
# # Google Gemma
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import torch

# tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it")
# model = AutoModelForCausalLM.from_pretrained("google/gemma-7b-it",
#                                             load_in_8bit=True,
#                                               device_map='auto',
#                                               torch_dtype=torch.float16,
#                                               low_cpu_mem_usage=True,
#                                               )

In [119]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,
    max_new_tokens=300,
    do_sample=True,
    temperature=0.8,
    top_k=50,
    top_p=0.95
)
local_llm = HuggingFacePipeline(pipeline = pipe)

In [126]:
from langchain.prompts import PromptTemplate
template = '''<|system|>Use the following pieces of information to answer the user's question in a concise manner.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
If answer is provided in the context, also report the Page Number.</s>

<|user|>
Context: {context}
    Question: {question}</s>
<|assistant|>
'''
prompt = PromptTemplate(input_variables=["context", "question"], template=template)

In [127]:
# # prompt Template for Gemma
# from langchain.prompts import PromptTemplate
# template = '''You are a helpful, respectful, and honest assistant.
# Always answer as helpfully as possible using the context text provided.
# Your answers should only answer the question once and not have any text after the answer is done.

# If a question does not make sense or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. Just say I don't know

# BEGIN CONTEXT
# {context}
# END CONTEXT

# Your turn:
# Question: {question}
# '''
# prompt = PromptTemplate(input_variables=["context", "question"], template=template)

In [153]:
question="How many extractions are there in the turbine?"

In [154]:
# docs_result = retriever.get_relevant_documents(question)
# docs_text = "\n".join([f"Page Number:{doc.metadata['page']}\n{doc.page_content}" for doc in docs_result])
# query = prompt.format(question=question, context=docs_text)
# chat = [
#     { "role": "user", "content": query },
# ]
# tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

In [155]:
def resolve_query(question):
    docs_result = retriever.get_relevant_documents(question)
    docs_text = "\n".join([f"Page Number:{doc.metadata['page']}\n{doc.page_content}" for doc in docs_result])
    query = prompt.format(question=question, context=docs_text)
    print(query)
#     chat = [
#          { "role": "user", "content": query },
#     ]
#     query = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
    ans = local_llm(query)
    return docs_text, ans


In [156]:
print(resolve_query(question)[1])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<|system|>Use the following pieces of information to answer the user's question in a concise manner.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
If answer is provided in the context, also report the Page Number.</s>

<|user|>
Context: Page Number:42
inner cylinder, IP cylinder and LP cylinder(s) sh all be horizontally split. ii) Steam turbine shall be complete with  casings, rotors, bearings, temperature  detectors, couplings, steam gland seal, roto r turning gear preferably hydraulic type and having local/rem ote operation facility, hand barring gear, emergency stop and control valves and reheat stop a nd control valves with their servomotors, removable type steam strainers for st art up and normal operation upstream of emergency stop an reheat stop valves, cr ossover/ cross around piping and electric motors for all its integral subsystems.  Steam strainers integral with main steam emergency stop and reheat stop valves are also acceptabl

In [27]:
import pandas
df = pandas.read_csv("https://docs.google.com/spreadsheets/d/1OqFAjz2Kz5KGjjIPsnue6tjsIZVj6zyqVVJVQ9a58MM/export?format=csv")

In [28]:
df.tail()

,Q. No.,Section,Question,Chat gpt-C- Expected answer,Zephyr 7B beta Answers at temp=0.8,Zephyr 7B beta Answers,Wizard 7B Answers
296,297,2.2.4.1(xii-b),What maintenance facilities are required for t...,Hinged access doors shall be provided in the h...,NaN,Page Number: 137\n \n Maintenance facilities r...,Answer:The following maintenance facilities ar...
297,298,2.2.4.1(xiii),What firefighting facilities are required for ...,Permanent firefighting facilities along with t...,NaN,"Permanent firefighting facilities, including s...",The firefighting facilities required for the a...
298,299,2.2.4.2(ii),Where should the condensate from the SCAPH be ...,Condensate from the SCAPH shall be led to the ...,NaN,Page Number: 210\n Answer: The condensate from...,The condensate from the SCAPH can be led to th...
299,300,2.2.4.2(v),What facility should be provided for the SCAPH?,Facility shall be provided for complete isolat...,NaN,Page Number:138\n Facility should be provided ...,Answer: Based on the given technical specifica...
300,301,2.2.4.2(vi),Under what conditions should the SCAPH be in s...,SCAPH should be in service only during startup...,NaN,The SCAPH (Steam Coal Ash Preheater) should be...,The SCAPH should be in service only during sta...


In [31]:
from tqdm import tqdm
from time import sleep
context = []
zephyr_answers = []
for row in tqdm(df.iterrows()):
    if row[0]>220:
        cont, answer = resolve_query(row[1]['Question'])
        context.append(cont)
        zephyr_answers.append(answer)

0it [00:00, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
222it [00:39,  5.69it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
223it [01:37,  1.82it/s]

...

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
299it [1:12:32, 44.45s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
300it [1:12:50, 36.48s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
301it [1:13:25, 14.64s/it]


In [ ]:
zephyr_series = pandas.Series(zephyr_answers, name="Zephyr 7B beta Answers at temp=0.8", index=range(len(zephyr_answers)))
context_series = pandas.Series(context, name="Context Retrival",index=range(len(context)))

In [ ]:
updated_df = pandas.concat([df,context_series,zephyr_series], axis=1)
updated_df

In [ ]:
updated_df.to_csv("answers.csv")

In [ ]:
updated_df.loc[0,"Zephyr 7B beta Answers"]

In [ ]:
print(resolve_query("What is the capacity of each unit in the thermal power project?")[1])